# KNN for Credit Decisions and Profit Maximization

*Auto-generated on 2025-11-12T09:28:25*

## 0. Environment setup (optional)
If you get errors reading `.xls`, run the cell below to install dependencies. Comment it out if you don't need it.

In [1]:
# Optional: install extra dependencies if needed
# Use only if you get import errors for xlrd or imbalanced-learn
# %pip install -q xlrd imbalanced-learn

## 1. Executive Summary (objective + deliverables)
**Objective.** Train and validate a supervised KNN classifier to estimate applicants' default probability and make approve/decline decisions that **maximize expected profit** under a business cost/benefit matrix.  
**Deliverables.** 1) KNN model encapsulated in a reproducible pipeline, 2) operating threshold optimized for utility, 3) holdout test evaluation with technical metrics and **business utility**, 4) deployment and monitoring guidelines.  
**Success criterion.** Beat baselines (approve all / approve none) in net utility; keep risk metrics aligned with policy (e.g., minimum TPR on a priority segment).  
**Key assumptions.** Use case: credit decision for existing customers (historical information available) with target "default next period." Costs and benefits are provided by the business (or scenario-based here).  
**Limitations.** KNN is sensitive to scaling, dimensionality, and inference latency; mitigated via preprocessing, k selection, and production controls.

## 2. Imports

In [2]:
import os
import json
import math
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, classification_report, brier_score_loss
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.calibration import calibration_curve
import joblib

pd.set_option("display.max_columns", 100)
print("Versions -> pandas:", pd.__version__)

Versions -> pandas: 2.2.3


## 3. Configuration (paths and business parameters)
Update `DATA_PATH` or `GITHUB_RAW_URL` if needed. Business costs/benefits are scenario placeholders; replace with real values when available.

In [ ]:
# Data source configuration
DATA_PATH = "default of credit card clients.xls"  # local path
GITHUB_RAW_URL = ""  # e.g., "https://raw.githubusercontent.com/<user>/<repo>/<branch>/path/to/file.xls"

# Target and basic options
TARGET_CANDIDATES = [
    "default.payment.next.month", 
    "default_payment_next_month", 
    "Y", "DEFAULT", "DEFAULT_NEXT_MONTH"
]

ID_CANDIDATES = ["ID", "Id", "id"]

# Business utility parameters (placeholder scenario). Replace with real values.
# Utility = TP*B_TP - FP*C_FP - FN*C_FN + TN*B_TN
BUSINESS_PARAMS = {
    "B_TP": 1.0,   # benefit of approving a non-defaulter
    "C_FP": 5.0,   # cost of approving a defaulter
    "C_FN": 0.3,   # opportunity cost of rejecting a good customer
    "B_TN": 0.0    # benefit of rejecting a defaulter (could be 0 or small positive)
}

RANDOM_STATE = 2025

## 4. Data loading and sanitation

In [6]:
def _standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Normalize column names: lower, underscores, no dots, no spaces
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.replace(r"[^\w\s\-\.]", "", regex=True)
          .str.replace(r"\s+", "_", regex=True)
          .str.replace(r"\.+", "_", regex=True)
          .str.lower()
    )
    return df

def _find_target(df: pd.DataFrame, candidates=TARGET_CANDIDATES) -> str:
    cols = set(df.columns)
    for c in candidates:
        c1 = c.lower().replace(".", "_")
        if c1 in cols:
            return c1
    raise KeyError(f"Target not found. Candidates: {candidates}. Available: {list(df.columns)[:10]} ...")

def _find_id(df: pd.DataFrame, candidates=ID_CANDIDATES) -> str | None:
    cols = set(df.columns)
    for c in candidates:
        c1 = c.lower()
        if c1 in cols:
            return c1
    return None

def load_dataset(path: str = None, url: str = None) -> pd.DataFrame:
    if url and url.strip():
        src = url.strip()
    elif path and os.path.exists(path):
        src = path
    else:
        raise FileNotFoundError("No valid data source. Set DATA_PATH or GITHUB_RAW_URL.")

    ext = os.path.splitext(src)[1].lower()
    if ext in [".xls", ".xlsx"]:
        # Try reading excel with fallback engines
        try:
            df = pd.read_excel(src, engine="xlrd")  # xls needs xlrd
        except Exception:
            try:
                df = pd.read_excel(src)  # try default/openpyxl for xlsx
            except Exception as e:
                raise RuntimeError(f"Failed to read Excel: {e}")
    elif ext in [".csv"]:
        df = pd.read_csv(src)
    else:
        # Try read_excel by default; if fails, try read_csv
        try:
            df = pd.read_excel(src)
        except Exception:
            df = pd.read_csv(src)

    df = _standardize_columns(df)
    return df

df_raw = load_dataset(DATA_PATH, GITHUB_RAW_URL)
print("Raw shape:", df_raw.shape)
display(df_raw.head())

RuntimeError: Failed to read Excel: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

## 5. Basic EDA and target preparation

In [ ]:
# Identify target and ID
target_col = _find_target(df_raw)
id_col = _find_id(df_raw)

print("Target column:", target_col, "| ID column:", id_col)

# Remove ID if present
df = df_raw.copy()
if id_col is not None and id_col in df.columns:
    df = df.drop(columns=[id_col])

# Ensure binary target is int {0,1}
if df[target_col].dtype != int and df[target_col].dtype != "int64":
    df[target_col] = df[target_col].astype(int)

y = df[target_col].values
X = df.drop(columns=[target_col])

print("Prepared shape -> X:", X.shape, " y:", y.shape)
print("Default rate (mean of target):", y.mean())
display(X.head())

## 6. Feature schema and preprocessing pipeline
One-hot for categorical codes (SEX, EDUCATION, MARRIAGE). Ordinal-like variables (PAY_0..PAY_6) and continuous amounts are treated as numeric and scaled.

In [ ]:
# Candidate categorical variables (encoded as small integers but represent categories)
candidate_cats = [c for c in ["sex", "education", "marriage"] if c in X.columns]

# Numeric = all others
numeric_cols = [c for c in X.columns if c not in candidate_cats]

print("Categorical:", candidate_cats)
print("Numeric (first 10):", numeric_cols[:10], "... total:", len(numeric_cols))

preprocess = ColumnTransformer(transformers=[
    ("num", StandardScaler(with_mean=True, with_std=True), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), candidate_cats)
], remainder="drop")

## 7. Train/Validation/Test split

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y
)

# From train_val, carve validation set (20% of the original data -> val_size=0.25 of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=RANDOM_STATE, stratify=y_train_val
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

## 8. KNN model and hyperparameter search

In [ ]:
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("knn", knn)
])

param_grid = {
    "knn__n_neighbors": [5, 11, 21, 31],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2]  # Manhattan vs Euclidean
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="roc_auc",   # technical metric for model selection
    cv=cv,
    n_jobs=-1,
    refit=True,
    verbose=0
)

gs.fit(X_train, y_train)
print("Best params:", gs.best_params_)
print("Best CV ROC-AUC:", round(gs.best_score_, 4))

best_model = gs.best_estimator_
# Evaluate on validation set (probabilities)
y_val_proba = best_model.predict_proba(X_val)[:, 1]

print("Val ROC-AUC:", round(roc_auc_score(y_val, y_val_proba), 4))
print("Val PR-AUC:", round(average_precision_score(y_val, y_val_proba), 4))
print("Val Brier:", round(brier_score_loss(y_val, y_val_proba), 4))

## 9. Utility function and threshold optimization on validation

In [ ]:
def confusion_counts(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn, fp, fn, tp

def utility_from_counts(tn, fp, fn, tp, params=BUSINESS_PARAMS):
    return (tp * params["B_TP"]) - (fp * params["C_FP"]) - (fn * params["C_FN"]) + (tn * params["B_TN"])

def classify_with_threshold(proba, thr):
    return (proba >= thr).astype(int)

def find_tau_star(y_true, proba, params=BUSINESS_PARAMS, grid=None):
    if grid is None:
        grid = np.linspace(0.0, 1.0, 1001)
    best_thr, best_u = None, -np.inf
    for t in grid:
        yhat = classify_with_threshold(proba, t)
        tn, fp, fn, tp = confusion_counts(y_true, yhat)
        u = utility_from_counts(tn, fp, fn, tp, params)
        if u > best_u:
            best_u = u
            best_thr = t
    return best_thr, best_u

tau_star, U_val = find_tau_star(y_val, y_val_proba, BUSINESS_PARAMS)
print("Optimal threshold (tau*):", round(float(tau_star), 4), "| Val Utility:", round(float(U_val), 4))

# Baselines (on validation using ground truth counts)
n_default = int(y_val.sum())
n_nondefault = int((1 - y_val).sum())
U_approve_all = (n_nondefault * BUSINESS_PARAMS["B_TP"]) - (n_default * BUSINESS_PARAMS["C_FP"])
U_reject_all = (n_default * BUSINESS_PARAMS["B_TN"]) - (n_nondefault * BUSINESS_PARAMS["C_FN"])

print("Baseline (approve all):", round(float(U_approve_all), 4))
print("Baseline (reject all):", round(float(U_reject_all), 4))

## 10. Final evaluation on test set

In [ ]:
y_test_proba = best_model.predict_proba(X_test)[:, 1]
y_test_hat = classify_with_threshold(y_test_proba, tau_star)

tn, fp, fn, tp = confusion_counts(y_test, y_test_hat)
U_test = utility_from_counts(tn, fp, fn, tp, BUSINESS_PARAMS)

roc_auc = roc_auc_score(y_test, y_test_proba)
pr_auc = average_precision_score(y_test, y_test_proba)

print("Test ROC-AUC:", round(roc_auc, 4), "| Test PR-AUC:", round(pr_auc, 4))
print("Confusion matrix [tn, fp, fn, tp]:", [tn, fp, fn, tp])
print("Test Utility:", round(float(U_test), 4))

# Baselines on test
n_default_test = int(y_test.sum())
n_nondefault_test = int((1 - y_test).sum())
U_test_approve_all = (n_nondefault_test * BUSINESS_PARAMS["B_TP"]) - (n_default_test * BUSINESS_PARAMS["C_FP"])
U_test_reject_all = (n_default_test * BUSINESS_PARAMS["B_TN"]) - (n_nondefault_test * BUSINESS_PARAMS["C_FN"])

print("Baseline Test (approve all):", round(float(U_test_approve_all), 4))
print("Baseline Test (reject all):", round(float(U_test_reject_all), 4))

print("\nClassification report (thresholded):\n", classification_report(y_test, y_test_hat, digits=4))

## 11. Probability calibration check (no plots)

In [ ]:
# Calibration via Brier score (already printed for val); here for test:
brier = brier_score_loss(y_test, y_test_proba)
print("Test Brier score:", round(brier, 4))

# Optional: compute calibration curve points
prob_true, prob_pred = calibration_curve(y_test, y_test_proba, n_bins=10, strategy="uniform")
calibration_table = pd.DataFrame({"mean_predicted": prob_pred, "fraction_of_positives": prob_true})
display(calibration_table)

## 12. Local interpretability: nearest neighbors for one test applicant

In [ ]:
# We'll inspect neighbors for one test sample
sample_ix = 0
xq = X_test.iloc[[sample_ix]]
yq = y_test[sample_ix]

# Transform features
pre = best_model.named_steps["prep"]
knn_est = best_model.named_steps["knn"]
xq_t = pre.transform(xq)

# Distances and indices among the training set used by KNN
dists, idxs = knn_est.kneighbors(xq_t, n_neighbors=min(knn_est.n_neighbors, len(X_train)))

# Show neighbors (map indices to the training DataFrame index)
neighbors_info = []
for d, i in zip(dists[0], idxs[0]):
    # The indices returned correspond to the order used internally (fit on transformed X_train)
    # We'll reconstruct by transforming X_train to ensure alignment of indices
    # Note: KNN stores training samples internally; to fetch the original row, use the position i
    orig_row = X_train.iloc[i].to_dict()
    orig_row["__distance__"] = float(d)
    orig_row["__y_train__"] = int(y_train[i])
    neighbors_info.append(orig_row)

neighbors_df = pd.DataFrame(neighbors_info)
display(pd.DataFrame({
    "query_true_label": [int(yq)],
    "n_neighbors": [knn_est.n_neighbors]
}))
display(neighbors_df.sort_values("__distance__").head(10))

## 13. Export artifacts (pipeline and config)

In [ ]:
ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

PIPELINE_PATH = os.path.join(ARTIFACT_DIR, "knn_credit_pipeline.joblib")
CONFIG_PATH = os.path.join(ARTIFACT_DIR, "knn_credit_config.json")

joblib.dump({
    "model": best_model,
    "features_numeric": best_model.named_steps["prep"].transformers_[0][2],
    "features_categorical": best_model.named_steps["prep"].transformers_[1][2],
    "params": gs.best_params_,
}, PIPELINE_PATH)

with open(CONFIG_PATH, "w", encoding="utf-8") as f:
    json.dump({
        "BUSINESS_PARAMS": BUSINESS_PARAMS,
        "tau_star": float(tau_star),
        "random_state": RANDOM_STATE
    }, f, indent=2)

print("Saved pipeline ->", PIPELINE_PATH)
print("Saved config   ->", CONFIG_PATH)

## 14. Conclusions and next steps
- KNN performance reported with ROC/PR and **business utility**.
- Operating threshold selected via **utility maximization** on validation.
- Replace placeholder BUSINESS_PARAMS with real costs/benefits; re-run threshold search.
- For production, profile latency and memory; consider approximate neighbors or a more scalable model if needed.